# CIFAR10 with CNNs
Simple starter notebook to benchmark your own CNN with PyTorch on the CIFAR-10 dataset.

OBS.:

- The main code is basically done, so focus on training the models and searching for the best hyperparameters and architectures.
- You are not required to use this exact code or even the PyTorch library.
- It is recommended to use execution environments with GPU access (such as Google Colab), since larger models will take more time to train.
- Remember to document the history of your experiments and which results motivated the changes in subsequent experiments.

In [ ]:
#@title Libs

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics

from tqdm import tqdm

In [ ]:
#@title Dataset Setup

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

### Defining the CNN model
Here we adapt the LeNet-5 architecture introduced in [*Gradient-based learning applied to document recognition*](https://ieeexplore.ieee.org/document/726791), originally developed to classify handwritten digits.

![lenet5](https://www.researchgate.net/publication/359784095/figure/fig2/AS:11431281079624737@1660789284522/Example-of-a-CNN-LeNet-5-14-is-able-to-identify-handwritten-digits-for-zip-code.png)


The main change we made here is the number of in/out channels, kernel sizes, padding, etc (enabling better results). It has 2 convolutional layers followed by 2 pooling (subsampling) layers. After the conv. blocks, we pass the feature maps through 2 hidden fully-connected layers to get the most activated neuron with the softmax function. If you want a more detailed explanation, [check this link](https://www.datasciencecentral.com/lenet-5-a-classic-cnn-architecture/) or reach us on Discord :)

In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    # Defining convolution blocks
    self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

    # Defining pooling layer
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    # Defining fully-connected layers
    self.fc_hidden1 = nn.Linear(64 * 8 * 8, 120)
    self.fc_hidden2 = nn.Linear(120, 84)
    self.fc_output = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(nn.ReLU()(self.conv1(x)))
    x = self.pool(nn.ReLU()(self.conv2(x)))
    x = x.view(x.size(0), -1) # flatten to input into fc layers
    x = nn.ReLU()(self.fc_hidden1(x))
    x = nn.ReLU()(self.fc_hidden2(x))
    x = self.fc_output(x)
    return x

In [ ]:
#@title Defining metrics helper

def get_scores(targets, predictions):
    return {
        "accuracy": metrics.accuracy_score(targets, predictions),
        "balanced_accuracy": metrics.balanced_accuracy_score(targets, predictions),
        "precision": metrics.precision_score(targets, predictions, average="weighted"),
        "recall": metrics.recall_score(targets, predictions, average="weighted"),
        "f1_score": metrics.f1_score(targets, predictions, average="weighted")
    }

In [ ]:
#@title Hyperparameters

learning_rate = 0.001
num_epochs = 10
batch_size = 32

loss_function = nn.CrossEntropyLoss()

In [ ]:
#@title Loaders

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
#@title Training loop

# Build the model
cnn = CNN()
cnn.cuda()

# Setting optimizer up
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

# Early stopping setup
best_loss = float('inf')
patience = 5
patience_counter = 0

# Start training epochs loop
for epoch in tqdm(range(num_epochs)):
  epoch_loss = 0.0
  for i, (images, labels) in enumerate(train_loader):
    images = images.cuda()
    labels = labels.cuda()

    # Forward pass
    optimizer.zero_grad()
    outputs = cnn(images)

    # Backward pass
    loss = loss_function(outputs, labels)
    loss.backward()

    optimizer.step()

    epoch_loss += loss.item()

    if (i+1) % 1000 == 0:
      tqdm.write(f' Epoch {epoch + 1}/{num_epochs}, Step {i+1}/{len(train_dataset) // batch_size}, Loss: {loss}')

  # Loss avrg
  epoch_loss /= len(train_loader)
  tqdm.write(f' Epoch {epoch+1} average loss: {epoch_loss:.4f}')

  # Early stopping using loss
  if epoch_loss < best_loss:
    best_loss = epoch_loss
    patience_counter = 0
  else:
    patience_counter += 1
    if patience_counter >= patience:
      tqdm.write("Early stopping triggered.")
      break

In [ ]:
#@title Evaluate model (accuracy, precision, recall, etc)

cnn.eval()
predictions = []
labels = []
for images, label in test_loader:
  images = images.cuda()
  label = label.cuda()

  output = cnn(images)
  _, predicted = torch.max(output,1)

  predictions.extend(predicted.cpu().numpy())
  labels.extend(label.cpu().numpy())

scores = get_scores(labels, predictions)
print("Scores of your model\n", scores)

# You can change/optimize this as you want
- Automatic hyperparameters optimization (Optuna)
- Regularization techniques (tip: try dropout)
- Use other convolution combinations (e.g. ResNet blocks)
- Validation set to track metrics during epochs
- Transform input data
- ...